## Auto mode (single uav)

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [2]:
gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_home = ENUPose(5,5,0,45)
base_path = Plan.create_square_path(side_len=10, alt=5,heading=0)

color =Color.BLUE

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

gaz_config.add(base_path=base_path,base_home=base_home,color=color)
gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin= gra_origin)

qgc_config.add(base_path=base_path,base_home=base_home,color=color)
qgc_config.show()

## No Visualizer

In [5]:
novis_config = ConfigVis[int]()
novis_config.add_vehicle(1)

In [6]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [7]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[gaz],
	gcs_sysids={'GCS': [1]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	verbose=2,
)
orac = simulator.launch()

🖥️ Gazebo launched for realistic simulation and 3D visualization.
🚀 ArduPilot SITL vehicle 1 launched (PID 2190215)
🚀 UAV logic for vehicle 1 launched (PID 2190217)
🚀 Proxy for vehicle 1 launched (PID 2190220)
[Errno 111] Connection refused sleeping
🚀 Starting Proxy 1


Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.


🔗 UAV logic 1 is connected to Oracle ⚪
✅ Heartbeat received
Logic 🧠 1: 🚀 lauching
Vehicle 1: ▶️ Plan Started: 📋 auto
Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
🚀 GCS GCS launched (PID 2190632)


Vehicle 1: ▶️ Step Started: clear uav missions


## Oracle checking

In [8]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

verb: 2
🔗 UAV logic 1 is connected
Vehicle 1: 🧹 Cleared previous mission
Vehicle 1: ✅ Step Done: clear uav missions
Vehicle 1: ✅ 7 waypoints read.
Vehicle 1: 🧭 Mission[0] → cmd: WAYPOINT, x: -35.3632831, y: 149.165169, z: 0.0, current: 0
Vehicle 1: 🧭 Mission[1] → cmd: TAKEOFF, x: -35.3632831, y: 149.165169, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[2] → cmd: WAYPOINT, x: -35.3633466, y: 149.1650911, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[3] → cmd: WAYPOINT, x: -35.3632831, y: 149.1650132, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[4] → cmd: WAYPOINT, x: -35.3632196, y: 149.1650911, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[5] → cmd: WAYPOINT, x: -35.3632831, y: 149.165169, z: 5.0, current: 0
Vehicle 1: 🧭 Mission[6] → cmd: LAND, x: -35.3632831, y: 149.165169, z: 0.0, current: 0
Vehicle 1: ✅ Sent mission item 0
Vehicle 1: ✅ Sent mission item 1
Vehicle 1: ✅ Sent mission item 2
Vehicle 1: ✅ Sent mission item 3
Vehicle 1: ✅ Sent mission item 4
Vehicle 1: ✅ Sent mission item 5
Vehicle 1: ✅ Sent mi

❎ Proxy 1 stopped.
